In [13]:
import sys
import zmq
import json
import pandas as pd
import os
from pandas_gbq import read_gbq
import plotly.express as px
import plotly.offline as pyo
import plotly.io as pio
import re

pio.renderers.default = 'notebook'  # or 'jupyterlab' if using JupyterLab
import warnings

# Suppress specific deprecation warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Libraries

In [14]:
NODE_DOMAIN_MAP = {
	"GABMKJM6I25XI4K7U6XWMULOUQIQ27BCTMLS6BYYSOWKTBUXVRJSXHYQ": "Stellar Development Foundation",
	"GCGB2S2KGYARPVIA37HYZXVRM2YZUEXA6S33ZU5BUDC6THSB62LZSTYH": "Stellar Development Foundation",
	"GCM6QMP3DLRPTAZW2UZPCPX2LF3SXWXKPMP3GKFZBDSF3QZGV2G5QSTK": "Stellar Development Foundation",
	"GAK6Z5UVGUVSEK6PEOCAYJISTT5EJBB34PN3NOLEQG2SUKXRVV2F6HZY": "SatoshiPay",
	"GBJQUIXUO4XSNPAUT6ODLZUJRV2NPXYASKUBY4G5MYP3M47PCVI55MNT": "SatoshiPay",
	"GC5SXLNAM3C4NMGK2PXK4R34B5GNZ47FYQ24ZIBFDFOCU6D4KBN4POAE": "SatoshiPay",
	"GCFONE23AB7Y6C5YZOMKUKGETPIAJA4QOYLS5VNS4JHBGKRZCPYHDLW7": "LOBSTR",
	"GCB2VSADESRV2DDTIVTFLBDI562K6KE3KMKILBHUHUWFXCUBHGQDI7VL": "LOBSTR",
	"GD5QWEVV4GZZTQP46BRXV5CUMMMLP4JTGFD7FWYJJWRL54CELY6JGQ63": "LOBSTR",
	"GA7TEPCBDQKI7JQLQ34ZURRMK44DVYCIGVXQQWNSWAEQR6KB4FMCBT7J": "LOBSTR",
	"GA5STBMV6QDXFDGD62MEHLLHZTPDI77U3PFOD2SELU5RJDHQWBR5NNK7": "LOBSTR",
	"GAAV2GCVFLNN522ORUYFV33E76VPC22E72S75AQ6MBR5V45Z5DWVPWEU": "Blockdaemon Inc.",
	"GAVXB7SBJRYHSG6KSQHY74N7JAFRL4PFVZCNWW2ARI6ZEKNBJSMSKW7C": "Blockdaemon Inc.",
	"GAYXZ4PZ7P6QOX7EBHPIZXNWY4KCOBYWJCA4WKWRKC7XIUS3UJPT6EZ4": "Blockdaemon Inc.",
	"GBLJNN3AVZZPG2FYAYTYQKECNWTQYYUUY2KVFN2OUKZKBULXIXBZ4FCT": "Public Node",
	"GCIXVKNFPKWVMKJKVK2V4NK7D4TC6W3BUMXSIJ365QUAXWBRPPJXIR2Z": "Public Node",
	"GCVJ4Z6TI6Z2SOGENSPXDQ2U4RKH3CNQKYUHNSSPYFPNWTLGS6EBH7I2": "Public Node",
	"GA7DV63PBUUWNUFAF4GAZVXU2OZMYRATDLKTC7VTCG7AU4XUPN5VRX4A": "Franklin Templeton",
	"GARYGQ5F2IJEBCZJCBNPWNWVDOFK7IBOHLJKKSG2TMHDQKEEC6P4PE4V": "Franklin Templeton",
	"GCMSM2VFZGRPTZKPH5OABHGH4F3AVS6XTNJXDGCZ3MKCOSUBH3FL6DOB": "Franklin Templeton",
	"GD6SZQV3WEJUH352NTVLKEV2JM2RH266VPEM7EH5QLLI7ZZAALMLNUVN": "Whalestack LLC",
	"GADLA6BJK6VK33EM2IDQM37L5KGVCY5MSHSHVJA4SCNGNUIEOTCR6J5T": "Whalestack LLC",
	"GAZ437J46SCFPZEDLVGDMKZPLFO77XJ4QVAURSJVRZK2T5S7XUFHXI2Z": "Whalestack LLC",
}

def filter_recent_data(df, minutes=5, timestamp_field="close_at"):
    df[timestamp_field] = pd.to_datetime(df[timestamp_field], utc=True)
    current_time = pd.Timestamp.utcnow()
    five_minutes_ago = current_time - pd.Timedelta(minutes=minutes)
    df = df[df[timestamp_field] >= five_minutes_ago]
    return df


def get_longest_leaders(df, filter_last_five_min=True):
    # Step 1: Filter last five minute worth of data if arg set to true
    if filter_last_five_min:
        filtered_df = filter_recent_data(df)
    else:
        filtered_df = df

    # Step 2: Assign row numbers based on closed_at
    filtered_df = filtered_df.sort_values('close_at')
    filtered_df['rn'] = range(1, len(filtered_df) + 1)

    # Step 3: Create a grouping identifier (grp)
    filtered_df['grp'] = filtered_df['rn'] - filtered_df.groupby('node_id')['rn'].transform(lambda x: x.rank(method='first'))

    # Step 4: Get start and end time for each group
    windowed_data = filtered_df.groupby(['node_id', 'grp']).agg(start_time=('close_at', 'min'),
                                                           end_time=('close_at', 'max')).reset_index()

    # Step 5: Calculate continuous_time and validator_frequency
    windowed_data['continuous_time'] = (windowed_data['end_time'] - windowed_data['start_time']).dt.total_seconds()
    result = windowed_data.groupby(['continuous_time', 'node_id']).size().reset_index(name='validator_frequency')

    # Step 6: Sort the results
    result = result.sort_values(by='continuous_time', ascending=False)
    result['home_domain'] = result['node_id'].map(NODE_DOMAIN_MAP)
    return result

def get_ledger_close_count_by_home_domain(df, filter_last_five_min=True):
    if filter_last_five_min:
        filtered_df = filter_recent_data(df)
    else:
        filtered_df = df

    mapped_df = filtered_df.copy()
    mapped_df['home_domain'] = filtered_df['node_id'].map(NODE_DOMAIN_MAP)
    if filter_last_five_min:
        mapped_df['minute'] = mapped_df['close_at'].dt.floor('min')
        mapped_df['minute'] = mapped_df['minute'].dt.strftime('%Y-%m-%d %H:%M')
        result_df = mapped_df.groupby(['home_domain', 'minute']).size().reset_index(name='count')
    else:
        mapped_df['day'] = mapped_df['close_at'].dt.date
        result_df = mapped_df.groupby(['home_domain', 'day']).size().reset_index(name='count')
    return result_df


def parse_operations(operations_str):
    # Extract the inner categories map and the total
    match = re.search(r'map\[categories:map\[(.*?)\] total:(\d+)\]', operations_str)
    if match:
        inner_map_str = match.group(1)
        total = int(match.group(2))
        
        # Convert the inner map to a dictionary, stripping spaces from keys
        inner_map = {key.strip(): int(value) for key, value in re.findall(r'([\w\s]+):(\d+)', inner_map_str)}

        # Add the total to the dictionary
        inner_map['total'] = total
        return inner_map
    return {}

def add_operation_type_cols(df):
    df['parsed_operations'] = df['operations'].apply(parse_operations)
    df['claimable_balances'] = df['parsed_operations'].apply(lambda x: x.get('claimable_balances', 0))
    df['account_creation'] = df['parsed_operations'].apply(lambda x: x.get('account_creation', 0))
    df['payments'] = df['parsed_operations'].apply(lambda x: x.get('payments', 0))
    df['offers_and_amms'] = df['parsed_operations'].apply(lambda x: x.get('offers_and_amms', 0))
    df['trust'] = df['parsed_operations'].apply(lambda x: x.get('trust', 0))
    df['sponsorship'] = df['parsed_operations'].apply(lambda x: x.get('sponsorship', 0))
    df['soroban'] = df['parsed_operations'].apply(lambda x: x.get('soroban', 0))
    df['other'] = df['parsed_operations'].apply(lambda x: x.get('other', 0))

    new_rows = []
    
    for _, row in df.iterrows():
        # Create a dictionary for each operation type
        for operation in ['claimable_balances', 'account_creation', 'payments', 
                          'offers_and_amms', 'trust', 'sponsorship', 
                          'soroban', 'other']:
            new_rows.append({
                'close_at': row['close_at'],
                'node_id': row['node_id'],
                'sequence_number': row['sequence_number'],
                'operation_type': operation,
                'parsed_operations': row['parsed_operations'],
                'count': int(row.get(operation))
            })

    new_df = pd.DataFrame(new_rows)
    return new_df


def get_operation_count_by_home_domain(df, filter_last_five_min=True):
    if filter_last_five_min:
        filtered_df = filter_recent_data(df)
    else:
        filtered_df = df

    mapped_df = filtered_df.copy()
    mapped_df['home_domain'] = filtered_df['node_id'].map(NODE_DOMAIN_MAP)
    if filter_last_five_min:
        mapped_df['minute'] = mapped_df['close_at'].dt.floor('min')
        mapped_df['minute'] = mapped_df['minute'].dt.strftime('%Y-%m-%d %H:%M')
        result_df = mapped_df.groupby(['home_domain', 'minute', 'operation_type']).agg({
            'count': 'sum'
        }).reset_index()
    else:
        mapped_df['day'] = mapped_df['close_at'].dt.date
        result_df = mapped_df.groupby(['home_domain', 'day', 'operation_type']).agg({
            'count': 'sum'
        }).reset_index()
    return result_df


## Plots

In [15]:
def write_plot_to_html(fig, filename, append=False):
    html_string = pyo.plot(fig, include_plotlyjs='cdn', output_type='div')
    
    if os.path.exists(f"{filename}.html") and append == True:
        with open(f"{filename}.html", "a") as f:
            f.write(html_string)
    else:
        with open(f"{filename}.html", "w") as f:
            f.write(html_string)

def plot_continuous_leader_chart(df, title, filename, append=False):
    fig = px.bar(
        df,
        x='continuous_time',
        y='validator_frequency',
        color='node_id',
        title=title,
        labels={'continuous_time': 'Continuous Time (seconds)', 'validator_frequency': 'Frequency'},
        text='home_domain'
    )

    # Update layout for grouping
    fig.update_layout(barmode='group')
    return fig


def plot_validator_frequency_chart(df, title, filename, x='', y='',labels={}, append=False):
    fig = px.bar(
        df,
        x=x,
        y=y,
        color='home_domain',
        title=title,
        labels=labels,
        barmode='stack'
    )

    if x == "minute":
        fig.update_xaxes(tickmode='array', tickvals=df['minute'], ticktext=df['minute'])
    else:
        fig.update_xaxes(tickmode='array', tickvals=df['day'], ticktext=df['day'])

    return fig

def plot_operation_type_frequency_chart(df, title, filename, x='', y='',labels={}, append=False):
    fig = px.bar(
        df,
        x=x,
        y=y,
        color='operation_type',
        title=title,
        labels=labels,
        barmode='stack'
    )

    if x == "minute":
        fig.update_xaxes(tickmode='array', tickvals=df['minute'], ticktext=df['minute'])
    else:
        fig.update_xaxes(tickmode='array', tickvals=df['day'], ticktext=df['day'])

    return fig

## Data Sources

In [16]:
def get_full_data_from_hubble():
    nodes = NODE_DOMAIN_MAP.keys()
    nodes_string = ",".join(f"'{item}'" for item in nodes)
    query = f"""
      SELECT
        hl.node_id as node_id,
        hl.closed_at AS close_at
      FROM crypto-stellar.crypto_stellar.history_ledgers AS hl
      WHERE hl.closed_at BETWEEN '2024-01-01 00:00:00 UTC' AND '2025-01-01 00:00:00 UTC'
      AND hl.node_id in ({nodes_string})
    """
    full_df = read_gbq(query, project_id='crypto-stellar')
    full_df['close_at'] = full_df['close_at'].dt.tz_localize(None)
    full_df['close_at'] = pd.to_datetime(full_df['close_at'])
    return full_df

def get_full_data_from_CSV_files():
    df = pd.read_csv("history_data.csv")
    df.rename(columns={'close_time': 'close_at'}, inplace=True)
    df['close_at'] = pd.to_datetime(df['close_at'], unit='s')
    return df

def get_live_data_socket_from_stream():
    #  Socket to talk to server
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    
    print("Collecting validator info from pipeline ...")
    socket.connect("tcp://127.0.0.1:5555")
    socket.subscribe("")
    return socket

## Full history Analytics

In [17]:
full_df = get_full_data_from_CSV_files()
full_data_figs = []
longest_leaders_result_df = get_longest_leaders(full_df, filter_last_five_min=False)
full_data_figs.append(plot_continuous_leader_chart(longest_leaders_result_df[longest_leaders_result_df["continuous_time"] >= 10], title="Nodes leading for greater than equal to 10 seconds", filename="graph", append=False))

validator_count_by_home_domain_df = get_ledger_close_count_by_home_domain(full_df, filter_last_five_min=False)
full_data_figs.append(plot_validator_frequency_chart(validator_count_by_home_domain_df, title="Validator count by day", filename="graph", x="day", y="count",labels={"day": "Timestamp by day", "count": "Closing validator count"}, append=True))

operation_count_by_home_domain_df = get_operation_count_by_home_domain(add_operation_type_cols(full_df), filter_last_five_min=False)
full_data_figs.append(plot_operation_type_frequency_chart(operation_count_by_home_domain_df[operation_count_by_home_domain_df["home_domain"] == "Stellar Development Foundation"], title="Operation count by day for SDF validators", filename="graph", x="day", y="count",labels={"day": "Timestamp by day", "count": "Operation type count"}, append=True))

## TODO: Add plot for operation count
## TODO:Add chi-square test

## Live data Analytics

In [ ]:
cur_data_vals = []
socket = get_live_data_socket_from_stream()

while True:
    live_data_figs = []
    message = socket.recv()
    json_object = json.loads(message)
    json_formatted_str = json.dumps(json_object, indent=2)

    cur_data_vals.append(json_object)
    cur_df = pd.DataFrame(cur_data_vals)
    cur_df.rename(columns={'close_time': 'close_at'}, inplace=True)
    cur_df['close_at'] = pd.to_datetime(cur_df['close_at'], unit='s')

    cur_res_df = get_longest_leaders(cur_df)
    live_data_figs.append(plot_continuous_leader_chart(cur_res_df, title="Nodes leading for last 5 minutes", filename="graph", append=True))

    cur_res_df = get_ledger_close_count_by_home_domain(cur_df)
    live_data_figs.append(plot_validator_frequency_chart(cur_res_df, title="Validator count by minute", filename="graph", x="minute", y="count",labels={"minute": "Timestamp by minutes", "count": "Closing validator count"}, append=True))

    append = False
    for fig in full_data_figs + live_data_figs:
        write_plot_to_html(fig, filename="graph", append=append)
        append = True


## Chi Square tests

In [22]:
from __future__ import division

'''
This test and code was motivated by 
http://stats.stackexchange.com/questions/100838/autocorrelation-of-discrete-time-series
and
http://stats.stackexchange.com/questions/73084/analysis-of-temporal-patterns/73170#73170
as described in
P. C. O'Brien and P. J. Dyck. A runs test based on run lengths. Biometrics, pages 237-244, 1985.
'''

from collections import Counter, defaultdict
import sys
import math
from scipy.stats import chi2

def weighted_variance(counts):
    avg = 0

    for length, count in counts.items():
        avg += count * length

    counts_only = counts.values()

    avg /= sum(counts_only)

    var = 0

    for length, count in counts.items():
        var += count * math.pow((length - avg),2)

    try:
        var /= sum(counts_only) - 1
    except:
        #var = 0
        raise Exception("Division by zero due to too few counts!")

    return var

def runs_test(input, path = True):
    '''
    You can pass a path or a dictionary of runs lengths
    path_passed = True states that you pass a path
    '''

    if path == True:
        counter = 1
        same = True
        cats = defaultdict(lambda : defaultdict(int))

        for i, elem in enumerate(input):
            #print elem, i
            if i == len(input) - 1:
                cats[elem][counter] += 1
                break

            if input[i+1] == elem:
                same = True
                counter += 1
            else:
                cats[elem][counter] += 1
                counter = 1
    else:
        cats = input

    x2 = 0
    df = 0
    nr_elem = len(cats.keys())
    fail_cnt = 0

    for elem in cats.keys():
        ns = sum([x*y for x,y in cats[elem].items()])
        rs = sum(cats[elem].values())

        #at the moment elements that have the following limitations get ignored
        #one could also think about throwing an exception here and stopping the calculation
        if len(cats[elem].keys()) == 1 or rs == 1 or (ns-rs) == 1:
            #print "Category '%s' has only one run length or only one run or ns-rs equals one! Sorry I will ignore it!" % elem
            fail_cnt += 1
            continue

        #print rs
        ss = weighted_variance(cats[elem])
        #print ss
        cs = (pow(rs,2)-1)*(rs+2)*(rs+3) / (2*rs*(ns-rs-1)*(ns+1))
        #print cs
        vs = cs * ns * (ns-rs) / (rs*(rs+1))

        x2 += ss * cs
        #print x2
        df += vs

    #note that this is kind-of a hack, you can adapt this as wanted
    if nr_elem - fail_cnt < 2:
        raise Exception("I ignored too many categories of this sequences! Sorry can't do the test!")

    if x2 == 0 or df == 0:
        raise Exception("x2 or df are zero, this really shouldn't happen!")
    pval = chi2.sf(x2,df)
    return pval

test_df = full_df.copy()
test_df['home_domain'] = test_df['node_id'].map(NODE_DOMAIN_MAP)
pval = runs_test(test_df.home_domain.values)
print(pval)

0.9053258978438159
